In [1]:
import geopandas as gpd
from shapely import LineString, Polygon, Point
import shapely
from shapely import ops
import matplotlib.pyplot as plt
from CPP_Planner import CPP_Algorithms, CPP_Planner_Kit, CPP_Algorithm_Optimizers
from shapely import affinity
from skimage.measure import approximate_polygon
import warnings
import math

In [2]:
all_road = gpd.read_file('test_Load_Shp/shp_file/村地和道路/村地道路.shp')
# all_land = gpd.read_file('test_Load_Shp/shp_file/村1地_全区.shp')
all_land = gpd.read_file('test_Load_Shp/shp_file/村地和道路/村地地块区域.shp')
all_road

In [3]:
_, ax = plt.subplots()
all_road.plot(ax=ax, color='b')
all_land.plot(ax=ax, color='orange')
ax.set_aspect('equal')

In [4]:
single_land = CPP_Planner_Kit.get_single_shp(all_land, 0)
single_land.plot()

In [5]:
split_polygon = CPP_Planner_Kit.split_polygon_by_largest_area(single_land.geometry.iloc[0], tolerance=0.05)
_, ax = plt.subplots()
for polygon in split_polygon:
    ax.plot(*polygon.exterior.xy)


In [6]:
# 旋转
single_polygon = split_polygon[0]
single_polygon_angle = CPP_Planner_Kit.get_land_MABR_angle(single_polygon)
single_polygon = affinity.rotate(single_polygon, -single_polygon_angle, origin='centroid')
_, ax = plt.subplots()
ax.plot(*single_polygon.exterior.xy)
ax.set_aspect('equal')

In [7]:
# 进行基础的路径规划，此时不用地头
single_polygon_regen = gpd.GeoDataFrame(geometry=[single_polygon], crs=all_land.crs)
path, headland = CPP_Algorithms.scanline_algorithm_single_with_headland(single_polygon_regen, 1.45, False, 'none')

In [8]:
_, ax = plt.subplots()
path.plot(ax=ax, color='y')
ax.plot(*single_polygon.exterior.xy, color='b')

In [9]:
# 假定一个地头宽度
headland_width = 9

In [10]:
# 检查当前路径和地块边界的交点
# 左侧
path_left = path.geometry
intersect_points = []
for line in path_left:
    # print(line.coords[0][0])
    if line.coords[0][0] < line.coords[-1][0]:
        left_point, right_point = line.coords[0], line.coords[-1]
    else:
        left_point, right_point = line.coords[-1], line.coords[0]
    # 因为是左侧，仅保留左侧很小的一部分，用于检测相交的点
    # temp_detection_line = LineString((left_point, (left_point[0] + 1, left_point[1])))
    # intersect_points.append(temp_detection_line.intersection(single_polygon))
    intersect_points.append(left_point)
intersect_points

In [11]:
line = LineString(intersect_points)
headland_edge = affinity.translate(line, xoff=headland_width)
# 制作出地头
_, ax = plt.subplots()
ax.plot(*line.xy, linewidth=5)
ax.plot(*headland_edge.xy, color='b')
ax.set_aspect('equal')

In [12]:
combined_polygon = list(line.coords)
combined_polygon += list(reversed(headland_edge.coords))
combined_polygon = Polygon(combined_polygon)
plt.plot(*combined_polygon.exterior.xy)

In [13]:
# 显示所有的结果
_, ax = plt.subplots()
ax.plot(*single_polygon.exterior.xy, color='b')
ax.plot(*combined_polygon.exterior.xy, color='gray')
ax.set_aspect('equal')

In [14]:
temp_path, temp_headland = CPP_Algorithms.scanline_algorithm_single_with_headland(single_polygon_regen, 1.45, False, 'both', headland_width)
_, ax = plt.subplots()
temp_headland.plot(ax=ax, color='gray')
ax.plot(*single_polygon.exterior.xy, color='b')

In [15]:
# 测试代码
temp_path_2, temp_headland_2 = CPP_Algorithms.scanline_algorithm_single_with_headland_2(
    single_polygon_regen, 1.45, False, 'both', headland_width
)
fig, ax = plt.subplots(dpi=300)
ax.plot(*single_polygon.exterior.xy, color='b')
temp_path_2.plot(ax=ax, color='y', linewidth=0.3)
temp_headland_2.plot(ax=ax, color='#DD0000')

In [16]:
single_polygon_2 = split_polygon[1]
single_polygon_angle_2 = CPP_Planner_Kit.get_land_MABR_angle(single_polygon_2)
single_polygon_2 = affinity.rotate(single_polygon_2, -single_polygon_angle_2, origin='centroid')
single_polygon_regen_2 = gpd.GeoDataFrame(geometry=[single_polygon_2], crs=all_land.crs)
temp_path_3, temp_headland_3 = CPP_Algorithms.scanline_algorithm_single_with_headland_2(
    single_polygon_regen_2, 1.45, False, 'both', headland_width
)
fig, ax = plt.subplots()
ax.plot(*single_polygon_2.exterior.xy, color='b')
temp_path_3.plot(ax=ax, color='y', linewidth=0.3)
temp_headland_3.plot(ax=ax, color='red')

In [20]:
all_headland = []
for i in temp_headland_2.geometry[0].geoms:
    all_headland.append(i)
for i in temp_headland_3.geometry[0].geoms:
    all_headland.append(i)
all_headland

In [21]:
_, ax = plt.subplots()
for poly in all_headland:
    ax.plot(*poly.exterior.xy)